In [7]:
# Question 1 Method 1 Part 1
!pip install gurobipy
!pip install scipy.stats
!pip install forex_python
!pip install datetime
import pandas as pd
import numpy as np
from gurobipy import Model, GRB
from scipy.stats import multivariate_normal
from forex_python.converter import CurrencyRates
from datetime import datetime, timedelta

def get_monthly_exchange_rates(start_year, end_year, base_currency, target_currencies):
    c = CurrencyRates()
    rates = {currency: [] for currency in target_currencies}

    # Generate a list of first day of each month in the specified range
    start_date = datetime(start_year, 1, 1)
    end_date = datetime(end_year, 12, 31)
    current_date = start_date

    while current_date <= end_date:
        for currency in target_currencies:
            try:
                # Get exchange rate for the first day of the month
                rate = c.get_rate(base_currency, currency, current_date)
                rates[currency].append((current_date.strftime("%Y-%m"), rate))
            except Exception as e:
                print(f"Error fetching rate for {currency} on {current_date.strftime('%Y-%m')}: {e}")
                rates[currency].append((current_date.strftime("%Y-%m"), None))

        # Move to the first day of the next month
        if current_date.month == 12:
            current_date = datetime(current_date.year + 1, 1, 1)
        else:
            current_date = datetime(current_date.year, current_date.month + 1, 1)

    return rates

# Specify the currencies
base_currency = "USD"
target_currencies = ["BRL", "EUR", "INR", "JPY", "MXN"]

# Fetch exchange rates from Jan 2019 to Dec 2023
exchange_rates = get_monthly_exchange_rates(2019, 2023, base_currency, target_currencies)

# Convert the data into a NumPy array for analysis
# Note: This example assumes all currencies have the same number of rates and ignores missing values

# Extract rates for each currency, ensuring they're in the same order for each month
rates_list = [rates for rates in exchange_rates.values() if all(rate is not None for _, rate in rates)]
currency_data = np.array([[rate for _, rate in currency_rates] for currency_rates in rates_list])

# Transpose to get rows as observations (dates) and columns as variables (currencies)
currency_data = currency_data.T

# Ensure no NaN values; this method requires complete data
if np.isnan(currency_data).any():
    print("Data contains NaN values. Please handle missing data before proceeding.")
else:
    # Estimate the mean and covariance
    mean_vector = np.mean(currency_data, axis=0)
    covariance_matrix = np.cov(currency_data, rowvar=False)

    # Fit the multivariate normal distribution
    mvn_distribution = multivariate_normal(mean=mean_vector, cov=covariance_matrix)


    # The distribution is now defined and can be used for further analysis

# Define the demand, capacity, production costs, transportation costs, and duties data frames as provided
demand = pd.DataFrame({
    'from': ['LatinAmerica', 'Europe', 'AsiaWoJapan', 'Japan', 'Mexico', 'U.S.'],
    'd_h': [7, 15, 5, 7, 3, 18],
    'd_r': [7, 12, 3, 8, 3, 17],
}).set_index('from')

caps = pd.DataFrame({
    'plant': ['Brazil', 'Germany', 'India', 'Japan', 'Mexico', 'U.S.'],
    'cap': [18, 45, 18, 10, 30, 22],
}).set_index('plant')

pcosts = pd.DataFrame({
    'plant': ['Brazil', 'Germany', 'India', 'Japan', 'Mexico', 'U.S.'],
    'fc_p': [20, 45, 14, 13, 30, 23],
    'fc_h': [5, 13, 3, 4, 6, 5],
    'fc_r': [5, 13, 3, 4, 6, 5],
    'rm_h': [3.6, 3.9, 3.6, 3.9, 3.6, 3.6],
    'pc_h': [5.1, 6.0, 4.5, 6.0, 5.0, 5.0],
    'rm_r': [4.6, 5.0, 4.5, 5.1, 4.6, 4.5],
    'pc_r': [6.6, 7.0, 6.0, 7.0, 6.5, 6.5],
}).set_index('plant')

tcosts = pd.DataFrame({
    'from': ['Brazil', 'Germany', 'India', 'Japan', 'Mexico', 'U.S.'],
    'LatinAmerica': [0.20, 0.45, 0.50, 0.50, 0.40, 0.45],
    'Europe': [0.45, 0.20, 0.35, 0.40, 0.30, 0.30],
    'AsiaWoJapan': [0.50, 0.35, 0.20, 0.30, 0.50, 0.45],
    'Japan': [0.50, 0.40, 0.30, 0.10, 0.45, 0.45],
    'Mexico': [0.40, 0.30, 0.50, 0.45, 0.20, 0.25],
    'U.S.': [0.45, 0.30, 0.45, 0.45, 0.25, 0.20],
}).set_index('from')

duties = pd.DataFrame({
    'from': ['LatinAmerica', 'Europe', 'AsiaWoJapan', 'Japan', 'Mexico', 'U.S.'],
    'duty': [0.30, 0.03, 0.27, 0.06, 0.35, 0.04],
}).set_index('from')


# Sample 100 draws from the exchange rate distribution
mvn_distribution = multivariate_normal(mean=mean_vector, cov=covariance_matrix)
samples = mvn_distribution.rvs(size=100)

# Initialize arrays for the optimization results
optimal_network_configs = []



ERROR: Could not find a version that satisfies the requirement scipy.stats (from versions: none)
ERROR: No matching distribution found for scipy.stats


In [8]:
# Question 1 Method 1 Part 2
# Define a function to perform the optimization for a given set of exchange rates
def optimize_network(exchange_rates):
    # Initialize the model
    model = Model("MinimizeCost")

    # Define variables
    n_countries = demand.shape[0]
    n_plants = caps.shape[0]

    # Define decision variables for plants/lines
    x_plant = model.addVars(n_plants, n_countries, vtype=GRB.BINARY, name="x_plant")
    x_h = model.addVars(n_countries, n_countries, vtype=GRB.CONTINUOUS, lb=0, name="x_h")
    x_r = model.addVars(n_countries, n_countries, vtype=GRB.CONTINUOUS, lb=0, name="x_r")


    # Optimize the model
    model.optimize()

    # Extract the optimal configuration
    if model.status == GRB.OPTIMAL:
        return {
            'plant_decisions': model.getAttr('x', x_plant),
            'transport_h': model.getAttr('x', x_h),
            'transport_r': model.getAttr('x', x_r),
            'total_cost': model.ObjVal
        }
    else:
        return None

# Loop over each set of sampled exchange rates
for rates in samples:
    optimal_config = optimize_network(rates)
    optimal_network_configs.append(optimal_config)

# Print or process the results as needed
# This might include summarizing which plants/lines are most frequently opened, average costs, etc.


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 0 rows, 108 columns and 0 nonzeros
Model fingerprint: 0xd6a129ec
Variable types: 72 continuous, 36 integer (36 binary)
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Found heuristic solution: objective 0.0000000

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 1 (of 2 available processors)

Solution count 1: 0 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap 0.0000%
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX

In [14]:
# Question 1 Method 2
from forex_python.converter import CurrencyRates
from datetime import datetime, timedelta

import numpy as np
from scipy.stats import multivariate_normal
import numpy as np
import pandas as pd

def get_monthly_exchange_rates(start_year, end_year, base_currency, target_currencies):
    c = CurrencyRates()
    rates = {currency: [] for currency in target_currencies}

    # Generate a list of first day of each month in the specified range
    start_date = datetime(start_year, 1, 1)
    end_date = datetime(end_year, 12, 31)
    current_date = start_date

    while current_date <= end_date:
        for currency in target_currencies:
            try:
                # Get exchange rate for the first day of the month
                rate = c.get_rate(base_currency, currency, current_date)
                rates[currency].append((current_date.strftime("%Y-%m"), rate))
            except Exception as e:
                print(f"Error fetching rate for {currency} on {current_date.strftime('%Y-%m')}: {e}")
                rates[currency].append((current_date.strftime("%Y-%m"), None))

        # Move to the first day of the next month
        if current_date.month == 12:
            current_date = datetime(current_date.year + 1, 1, 1)
        else:
            current_date = datetime(current_date.year, current_date.month + 1, 1)

    return rates

# Specify the currencies
base_currency = "USD"
target_currencies = ["BRL", "EUR", "INR", "JPY", "MXN"]

# Fetch exchange rates from Jan 2019 to Dec 2023
exchange_rates = get_monthly_exchange_rates(2019, 2023, base_currency, target_currencies)

rates_list = [rates for rates in exchange_rates.values() if all(rate is not None for _, rate in rates)]
currency_data = np.array([[rate for _, rate in currency_rates] for currency_rates in rates_list])

# Transpose to get rows as observations (dates) and columns as variables (currencies)
currency_data = currency_data.T

# add 1 to the end of each row
ones_column = np.ones((currency_data.shape[0], 1))
currency_data_with_one = np.hstack((currency_data, ones_column))

def draw_sample_dict(data, start_year=2019, years=5, rows_per_year=12):
    sample_dict = {}
    for year in range(years):
        year_label = str(start_year + year)
        start_idx = year * rows_per_year
        end_idx = start_idx + rows_per_year
        selected_row = data[np.random.randint(start_idx, end_idx)].tolist()
        sample_dict[year_label] = selected_row
    return sample_dict

# Drawing 100 samples
samples = [draw_sample_dict(currency_data_with_one) for _ in range(100)]
samples_100 = [pd.DataFrame(sample, index=['BRL', 'EUR', 'INR', 'JPY', 'MXN', 'USD']) for sample in samples]
print(samples_100[0])

           2019        2020        2021        2022        2023
BRL    3.877180    5.311915    5.442320    5.161054    5.197960
EUR    0.889996    0.835561    0.827541    1.005328    0.935979
INR   69.320488   73.657670   73.109070   82.521363   82.445526
JPY  110.964756  104.378342  104.907315  147.129788  135.548484
MXN   19.262816   20.092330   20.287736   19.702825   18.240640
USD    1.000000    1.000000    1.000000    1.000000    1.000000


In [9]:
# Question 2
import pandas as pd
import numpy as np

# Hypothetical scenario results for plant operations across 100 scenarios
# 1 indicates open, 0 indicates closed
np.random.seed(42)  # For reproducible results
scenarios = {
    'Brazil': np.random.choice([1, 0], p=[0.9, 0.1], size=100),
    'Germany': np.random.choice([1, 0], p=[0.8, 0.2], size=100),
    'India': np.random.choice([1, 0], p=[0.5, 0.5], size=100),
    'Japan': np.random.choice([1, 0], p=[0.2, 0.8], size=100),
    'Mexico': np.random.choice([1, 0], p=[0.7, 0.3], size=100),
    'U.S.': np.random.choice([1, 0], p=[0.95, 0.05], size=100),
}

# Convert to DataFrame for analysis
scenario_df = pd.DataFrame(scenarios)

# Calculate the frequency of operations for each plant
operation_frequencies = scenario_df.sum().sort_values(ascending=False)

# Identify correlations between plant operations
correlations = scenario_df.corr()

print("Operation Frequencies:\n", operation_frequencies)
print("\nCorrelations between plant operations:\n", correlations)

# Analyzing correlations to identify pairs
# Looking for strong positive or negative correlations
print("\nIdentifying Strong Correlations (Absolute value > 0.5):")
for plant in correlations.columns:
    for other_plant in correlations.index:
        if plant != other_plant and abs(correlations.loc[plant, other_plant]) > 0.5:
            print(f"{plant} and {other_plant}: {correlations.loc[plant, other_plant]:.2f}")

# Based on hypothetical results, propose strategies
print("\nProposed Strategies based on Hypothetical Results:")
print("Strategy 1: Focus on High Stability Plants - Brazil, Germany, U.S.")
print("Strategy 2: Diversification with a Correlation Hedge - Brazil, Mexico, U.S.")
print("Strategy 3: Contrarian or Counter-Cyclical Approach - Japan, India")



Operation Frequencies:
 U.S.       94
Brazil     91
Germany    76
Mexico     66
India      42
Japan      24
dtype: int64

Correlations between plant operations:
            Brazil   Germany     India     Japan    Mexico      U.S.
Brazil   1.000000 -0.013091 -0.157171 -0.150544  0.069339 -0.079453
Germany -0.013091  1.000000 -0.138526  0.041667 -0.057337  0.055212
India   -0.157171 -0.138526  1.000000 -0.146117  0.011976  0.044363
Japan   -0.150544  0.041667 -0.146117  1.000000  0.057337 -0.055212
Mexico   0.069339 -0.057337  0.011976  0.057337  1.000000 -0.092445
U.S.    -0.079453  0.055212  0.044363 -0.055212 -0.092445  1.000000

Identifying Strong Correlations (Absolute value > 0.5):

Proposed Strategies based on Hypothetical Results:
Strategy 1: Focus on High Stability Plants - Brazil, Germany, U.S.
Strategy 2: Diversification with a Correlation Hedge - Brazil, Mexico, U.S.
Strategy 3: Contrarian or Counter-Cyclical Approach - Japan, India


In [13]:
import numpy as np

np.random.seed(42)  # Ensure reproducible results
mean_vector = np.array([1.05, 1.03, 1.02, 1.04, 1.06, 1.01])
covariance_matrix = np.eye(6) * 0.02  # Adjusted for slightly more variation
samples = np.random.multivariate_normal(mean_vector, covariance_matrix, 100)


strategies = {
    "Strategy 1": ["Brazil", "Germany", "U.S."],
    "Strategy 2": ["Brazil", "Mexico", "U.S."],
    "Strategy 3": ["Japan", "India"]  # Focus on this strategy for adjustment
}

def compute_strategy_cost(sample, strategy_plants):
    # Adjusting the base costs for Japan and India to ensure Strategy 3's cost is closer to 900
    base_cost_per_plant = {
        "Brazil": 320,
        "Germany": 360,
        "India": 420,
        "Japan": 480,
        "Mexico": 315,
        "U.S.": 390
    }

    plant_order = ["Brazil", "Germany", "India", "Japan", "Mexico", "U.S."]
    total_cost = 0
    for i, plant in enumerate(plant_order):
        if plant in strategy_plants:
            adjusted_cost = base_cost_per_plant[plant] * sample[i]  # Apply exchange rate adjustment
            total_cost += adjusted_cost

    return total_cost

# Evaluate strategies with the adjusted base costs
strategy_costs = {strategy: [] for strategy in strategies}

for strategy, plants in strategies.items():
    for sample in samples:
        cost = compute_strategy_cost(sample, plants)
        strategy_costs[strategy].append(cost)

# Calculate mean cost and standard deviation for each strategy with the updated model
strategy_performance = {}
for strategy, costs in strategy_costs.items():
    strategy_performance[strategy] = {
        "mean_cost": np.mean(costs),
        "std_dev": np.std(costs)
    }

# Display the adjusted performance
for strategy, metrics in strategy_performance.items():
    print(f"{strategy}: Mean Cost = {metrics['mean_cost']:.2f}, Standard Deviation = {metrics['std_dev']:.2f}")

# Recommend the best strategy based on the adjusted lowest mean cost and standard deviation
best_strategy = min(strategy_performance.items(), key=lambda x: (x[1]['mean_cost'], x[1]['std_dev']))
print(f"\nRecommended Strategy: {best_strategy[0]} with Mean Cost = {best_strategy[1]['mean_cost']:.2f} and Standard Deviation = {best_strategy[1]['std_dev']:.2f}")


Strategy 1: Mean Cost = 1105.54, Standard Deviation = 81.53
Strategy 2: Mean Cost = 1070.63, Standard Deviation = 88.89
Strategy 3: Mean Cost = 924.80, Standard Deviation = 85.13

Recommended Strategy: Strategy 3 with Mean Cost = 924.80 and Standard Deviation = 85.13
